# Training PISA

$^*$: 意指詳見reference

In [ ]:
# 前面要先灌mmdetection
!pip install openmim
!mim install mmdet

## 1.Configure file

### 1.1.命名風格
MMDetection對於Configure File的命名風格如下：
{model}\_{backbone}\_{neck}\_{schedule}\_{dataset}

- {model}： model 種類，例如：faster_rcnn, mask_rcnn 等。
- {backbone}：Backbone 種類，例如：r50 (ResNet-50), x101 (ResNeXt-101) 等。
- {neck}：Neck 種類，例如：fpn, pafpn, nasfpn, c4 等。
- {schedule}： 訓練方案，選項是 1x、 2x、 20e 等。
> 1x 和 2x 分別代表 12 epoch 和 24 epoch，20e 在級聯 (Cascaded) 模型中使用，表示 20 epoch。
- {dataset}：資料集，例如：coco、 cityscapes、 voc_0712、 wider_face 等。

例如pisa_faster_rcnn_x101_32x4d_fpn_1x_coco.py的意思就是:
- 使用Prime sample attention(PISA)$^*$方案的FasterRCNN$^*$物件偵測模型 
- Backbone用的是ResNext101$^*$，每個cell用的是32個group做concatenate，並且寬度為4
- Neck用的是Feature Pyramid Networks(FPN)$^*$，在YOLOv4也有用
- 訓練上這邊用的是 12 epoch
- 訓練在coco dataset$^*$上

### 1.2基本內容

作者為了可以提高代碼複用率，所以 config 支持繼承的操作，通過 \_base\_ 變量來實現，\_base\_ 是一個 list 類型變量，裡面存放的是要繼承配置文件的路徑，任何配置文件都能往上追朔繼承以下四種類型的文件：
- 模型 (models)
- 資料集 (datasets)
- 訓練策略 (schedules)
- 運行時的默認配置 (default_runtime)

在運算過程中mmdetection會把多個config檔整合成一個，所以後面我們下載下來的基本pisa config檔只有一個

這邊看一下pisa_faster_rcnn_x101_32x4d_fpn_1x_coco.py 長怎樣：

In [ ]:
# 安裝mmdet時會順便安裝mmcv，是mm系列有關cv演算法的核心，其中包含config讀取
from mmcv import Config

In [1]:
# 沒有這份config檔請下載
!mim download mmdet --config pisa_faster_rcnn_x101_32x4d_fpn_1x_coco --dest .
cfg=Config.fromfile("pisa_faster_rcnn_x101_32x4d_fpn_1x_coco.py")

processing pisa_faster_rcnn_x101_32x4d_fpn_1x_coco...
  [####################################]  100%          
Successfully downloaded pisa_faster_rcnn_x101_32x4d_fpn_1x_coco-e4accec4.pth to /home/jovyan/course/pisa-simple-example
Successfully dumped pisa_faster_rcnn_x101_32x4d_fpn_1x_coco.py to /home/jovyan/course/pisa-simple-example


大致上包含資料、模型、訓練、測試、推論的每個設定

In [10]:
cfg.keys()

dict_keys(['model', 'dataset_type', 'data_root', 'img_norm_cfg', 'train_pipeline', 'test_pipeline', 'data', 'evaluation', 'optimizer', 'optimizer_config', 'lr_config', 'runner', 'checkpoint_config', 'log_config', 'custom_hooks', 'dist_params', 'log_level', 'load_from', 'resume_from', 'workflow', 'opencv_num_threads', 'mp_start_method', 'auto_scale_lr'])

#### 1.2.1.Model設定

Model裡面就把架構有系統的建構方式設定起來，也有有關訓練和測試的設定

In [15]:
cfg.model.keys()

dict_keys(['type', 'backbone', 'neck', 'rpn_head', 'roi_head', 'train_cfg', 'test_cfg'])

##### **一些非重點的設定**

**類型**

物件偵測模型類型，由於PISA演算法的核心是在roi_head的cnn blocks中實施self attention，所以與這個無關

但是在論文上主要有使用到FasterRCNN來做主架構更動

In [16]:
cfg.model.type

'FasterRCNN'

**Backbone模型**

同個物件偵測模型中的backbone模型可以換置，會影響圖片的embedding效果，我們這邊用論文中做得較好的ResNext模型。

In [18]:
cfg.model.backbone

{'type': 'ResNeXt',
 'depth': 101,
 'num_stages': 4,
 'out_indices': (0, 1, 2, 3),
 'frozen_stages': 1,
 'norm_cfg': {'type': 'BN', 'requires_grad': True},
 'norm_eval': True,
 'style': 'pytorch',
 'init_cfg': {'type': 'Pretrained',
  'checkpoint': 'open-mmlab://resnext101_32x4d'},
 'groups': 32,
 'base_width': 4}

**Neck模型**

Neck 模型來將對應不同scale投射到不同個輸出時使用，使用FPN(*ref)

In [23]:
cfg.model.neck

{'type': 'FPN',
 'in_channels': [256, 512, 1024, 2048],
 'out_channels': 256,
 'num_outs': 5}

**Region Proposal Network Head**

使用Region Proposal Network(RPN，詳見FasterRCNN論文) 以前面Neck的資訊做物件區域偵測的提案。

In [25]:
cfg.model.rpn_head

{'type': 'RPNHead',
 'in_channels': 256,
 'feat_channels': 256,
 'anchor_generator': {'type': 'AnchorGenerator',
  'scales': [8],
  'ratios': [0.5, 1.0, 2.0],
  'strides': [4, 8, 16, 32, 64]},
 'bbox_coder': {'type': 'DeltaXYWHBBoxCoder',
  'target_means': [0.0, 0.0, 0.0, 0.0],
  'target_stds': [1.0, 1.0, 1.0, 1.0]},
 'loss_cls': {'type': 'CrossEntropyLoss',
  'use_sigmoid': True,
  'loss_weight': 1.0},
 'loss_bbox': {'type': 'L1Loss', 'loss_weight': 1.0}}

##### **Region of Interest Head**

使用Region of Interest(ROI，詳見FasterRCNN論文)head 將最終預測算出。

*這邊就是PISA將Attention加進model的地方*

In [26]:
cfg.model.roi_head

{'type': 'PISARoIHead',
 'bbox_roi_extractor': {'type': 'SingleRoIExtractor',
  'roi_layer': {'type': 'RoIAlign', 'output_size': 7, 'sampling_ratio': 0},
  'out_channels': 256,
  'featmap_strides': [4, 8, 16, 32]},
 'bbox_head': {'type': 'Shared2FCBBoxHead',
  'in_channels': 256,
  'fc_out_channels': 1024,
  'roi_feat_size': 7,
  'num_classes': 80,
  'bbox_coder': {'type': 'DeltaXYWHBBoxCoder',
   'target_means': [0.0, 0.0, 0.0, 0.0],
   'target_stds': [0.1, 0.1, 0.2, 0.2]},
  'reg_class_agnostic': False,
  'loss_cls': {'type': 'CrossEntropyLoss',
   'use_sigmoid': False,
   'loss_weight': 1.0},
  'loss_bbox': {'type': 'SmoothL1Loss', 'loss_weight': 1.0, 'beta': 1.0}}}

#### 1.2.2.資料設定

- xxx_per_gpu: 一些GPU設定
- data.train/val/test: 訓練、驗證、測試集都有不同的設定 (下面以train來講解)
- data.train.type: 資料集類型，主要是標註的格式
- data.train.ann_file: 標註檔位置
- data.train.img_prefix: 圖檔資料夾位置
- data.train.pipeline: 用list方式列出pre-process會用的演算法，可能有normalize大小、翻轉、旋轉等等可以用

In [27]:
cfg.data

{'samples_per_gpu': 2,
 'workers_per_gpu': 2,
 'train': {'type': 'CocoDataset',
  'ann_file': 'data/coco/annotations/instances_train2017.json',
  'img_prefix': 'data/coco/train2017/',
  'pipeline': [{'type': 'LoadImageFromFile'},
   {'type': 'LoadAnnotations', 'with_bbox': True},
   {'type': 'Resize', 'img_scale': (1333, 800), 'keep_ratio': True},
   {'type': 'RandomFlip', 'flip_ratio': 0.5},
   {'type': 'Normalize',
    'mean': [123.675, 116.28, 103.53],
    'std': [58.395, 57.12, 57.375],
    'to_rgb': True},
   {'type': 'Pad', 'size_divisor': 32},
   {'type': 'DefaultFormatBundle'},
   {'type': 'Collect', 'keys': ['img', 'gt_bboxes', 'gt_labels']}]},
 'val': {'type': 'CocoDataset',
  'ann_file': 'data/coco/annotations/instances_val2017.json',
  'img_prefix': 'data/coco/val2017/',
  'pipeline': [{'type': 'LoadImageFromFile'},
   {'type': 'MultiScaleFlipAug',
    'img_scale': (1333, 800),
    'flip': False,
    'transforms': [{'type': 'Resize', 'keep_ratio': True},
     {'type': 'Rand

沒資料的話請下載資料

In [ ]:
!mkdir -p data/coco
!curl -L "https://public.roboflow.com/ds/teSUuBdtOy?key=MSfxYe5Fz3" > data/coco/roboflow.zip
!unzip -f data/coco/roboflow.zip -d data/coco
!rm -rf data/coco/roboflow.zip

In [1]:
# 可以先看看用法指南，不過蠻多的也容易混亂，可以參考後面簡單的使用方式
!python tools/train.py --help

usage: train.py [-h] [--work-dir WORK_DIR] [--resume-from RESUME_FROM]
                [--auto-resume] [--no-validate]
                [--gpus GPUS | --gpu-ids GPU_IDS [GPU_IDS ...] | --gpu-id
                GPU_ID] [--seed SEED] [--diff-seed] [--deterministic]
                [--options OPTIONS [OPTIONS ...]]
                [--cfg-options CFG_OPTIONS [CFG_OPTIONS ...]]
                [--launcher {none,pytorch,slurm,mpi}]
                [--local_rank LOCAL_RANK] [--auto-scale-lr]
                config

Train a detector

positional arguments:
  config                train config file path

optional arguments:
  -h, --help            show this help message and exit
  --work-dir WORK_DIR   the dir to save logs and models
  --resume-from RESUME_FROM
                        the checkpoint file to resume from
  --auto-resume         resume from the latest checkpoint automatically
  --no-validate         whether not to evaluate the checkpoint during training
  --gpus GPUS           (Depr

In [ ]:
# 最簡單來個config檔就結束了
!python tools/train.py train_config.py 

## Reference

* [PISA] Cao, Y., Chen, K., Loy, C. C., & Lin, D. (2020). Prime sample attention in object detection. In Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition (pp. 11583-11591).
* [ResNext] Xie, S., Girshick, R., Dollár, P., Tu, Z., & He, K. (2017). Aggregated residual transformations for deep neural networks. In Proceedings of the IEEE conference on computer vision and pattern recognition (pp. 1492-1500).
* [FPN] Lin, T. Y., Dollár, P., Girshick, R., He, K., Hariharan, B., & Belongie, S. (2017). Feature pyramid networks for object detection. In Proceedings of the IEEE conference on computer vision and pattern recognition (pp. 2117-2125).
* [COCO Dataset] Lin, T. Y., Maire, M., Belongie, S., Hays, J., Perona, P., Ramanan, D., ... & Zitnick, C. L. (2014, September). Microsoft coco: Common objects in context. In European conference on computer vision (pp. 740-755). Springer, Cham.